# Running

I love running. I also collect a lot of data about my runs through my Garmin watch. It's about time I did something interesting with it.

## Where to get the data from?

Garmin Connect and Strava are the two obvious places to retrieve my running history. Both give the user the option to bulk export their data history.

- Garmin - [https://www.garmin.com/en-US/account/datamanagement/exportdata/](https://www.garmin.com/en-US/account/datamanagement/exportdata/)
- Strava - [https://www.strava.com/athlete/delete_your_account](https://www.strava.com/athlete/delete_your_account) (don't be scared by the page name)

These need to be compiled so these requests can take a while. They both send a link to your email address for you to retrieve the data. For me, Strava took about 5 mins, Garmin took about 20 mins. After having a quick look through both folders, Strava seems a lot more intuitive to understand, so for now I'll focus on files from there.

## File types

There are several file types across the folders=

- `csv` - the activites summary is in this format
- `gpx` - routes and some activities are stored as this
- `gz` - most of the activities are in this format